In [1]:
import pysam

In [3]:
samfile = pysam.AlignmentFile("/sbgenomics/project-files/merged-tumor.bam", "rb")

Analysis of the first read

In [4]:
for read in samfile:
    first_read = read
    break

In [22]:
fields = [
    { "name": "QUERY NAME", "prop": "query_name" },
    { "name": "FLAG", "prop": "flag" },
    { "name": "REFERENCE ID", "prop": "reference_id" },
    { "name": "REFERENCE START", "prop": "reference_start" },
    { "name": "MAPPING QUALITY", "prop": "mapping_quality" },
    { "name": "CIGAR TUPLES", "prop": "cigartuples", "collection": True },
    { "name": "NEXT REFERENCE ID", "prop": "next_reference_id" },
    { "name": "NEXT REFERENCE START", "prop": "next_reference_start" },
    { "name": "TEMPLATE LENGTH", "prop": "template_length" },
    { "name": "QUERY SEQUENCE", "prop": "query_sequence" },
    { "name": "QUERY QUALITIES", "prop": "query_qualities", "collection": False }
]

for field in fields:
    print(">>> " + field['name'])
    
    data = getattr(first_read, field['prop'])
    
    if 'collection' in field and field['collection']:
        for e in data:
            print(e)
    else:
        print(data)

    print()
    
print(">>> TAGS")
for tag in first_read.get_tags():
    print(tag)

>>> QUERY NAME
C0HVYACXX120402:7:1207:5722:57044

>>> FLAG
1187

>>> REFERENCE ID
20

>>> REFERENCE START
9483248

>>> MAPPING QUALITY
27

>>> CIGAR TUPLES
(0, 76)

>>> NEXT REFERENCE ID
20

>>> NEXT REFERENCE START
9483381

>>> TEMPLATE LENGTH
209

>>> QUERY SEQUENCE
TTTTCAAACAGTATCTATGCCTGCCAAATGTGAACATATAAAAAAAAACCAGAATGTGCCATTCTGATTTAAACTG

>>> QUERY QUALITIES
array('B', [28, 28, 27, 29, 31, 30, 31, 31, 29, 31, 35, 30, 29, 31, 34, 30, 29, 23, 41, 32, 20, 30, 29, 34, 34, 29, 30, 31, 30, 30, 30, 33, 33, 26, 39, 12, 25, 19, 32, 30, 35, 28, 35, 33, 23, 33, 35, 36, 30, 38, 33, 41, 34, 35, 31, 33, 23, 30, 30, 36, 27, 32, 29, 34, 35, 41, 33, 31, 33, 29, 32, 32, 31, 31, 31, 34])

>>> TAGS
('XA', 'GL000217.1,-110754,76M,1;')
('BD', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN')
('MD', '76')
('RG', '1')
('BI', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN')
('NM', 0)
('MQ', 27)
('AS', 76)
('XS', 71)


Analysis of the flag of the first read

In [27]:
print(str(first_read.flag) + " = " + str(hex(first_read.flag)))

1187 = 0x4a3


0x4a3 = 0x400 + 0x080 + 0x020 + 0x002 + 0x001

0x001 => the read is paired in sequencing, no matter whether it is mapped in a pair

0x002 => the read is mapped in a proper pair (depends on the protocol, normally inferred during alignment)

0x020 => strand of the mate

0x080 => the read is the second read in a pair

0x400 => the read is either a PCR duplicate or an optical duplicate




Stats

In [23]:
unmappedReads = 0
qualZReads = 0
totalReads = 0

avgMappingQual = 0
avgNZMappingQual = 0

In [24]:
for read in samfile:
    if not(read.is_unmapped):
        unmappedReads += 1
        
    if read.mapping_quality == 0:
        qualZReads += 1
        
    totalReads += 1
    
    avgMappingQual += read.mapping_quality
    
    if read.mapping_quality != 0:
        avgNZMappingQual += read.mapping_quality
        
avgMappingQual /= totalReads
avgNZMappingQual /= totalReads - qualZReads

In [25]:
print('How many unmapped reads are in the file? ' + str(unmappedReads))
print()

print('Total number of reads: ' + str(totalReads))
print()

print('Number of reads with mapping quality 0: ' + str(qualZReads))
print()

print('Average mapping quality for all the reads: ' + str(avgMappingQual))
print()

print('Average mapping quality if reads with 0 mapp quality are filtered out: ' + str(avgNZMappingQual))

How many unmapped reads are in the file? 2903863

Total number of reads: 2921628

Number of reads with mapping quality 0: 126628

Average mapping quality for all the reads: 55.91380148328261

Average mapping quality if reads with 0 mapp quality are filtered out: 58.446986762075134


In [28]:
samfile.close()